# Работа с удаленным репозиторием


В основном работа происходит в команде, причем удобно хранить рабочую версию проекта на каком-то общем сервере. git предоставляет удобный инстремент для работы с удаленными репозиториями.

In [ ]:
# создадим репозиторий...
import os
if not os.path.isdir('folder'):
    os.mkdir('folder')
!git init folder

In [ ]:
# Напилним его содержимым...
with open('folder/file', 'w') as file:
    file.write('some text\n')
os.chdir('folder')
!git add file
!git commit -m "added file"

In [ ]:
# вернемся обратно
os.chdir('..')

## git remote

In [ ]:
!git clone folder folder_local

In [ ]:
import os
os.chdir('folder_local/')

Если набрать команду без флагов, то гит покажет, какие удаленные сервера мы сконфигурировали

In [ ]:
!git remote

Посмотрим, что за origin

- fetch -- откуда получаем
- push -- куда загружаем

In [ ]:
!git remote -v

У репозитория может быть несколько ремотов (что ничему не противоречит)

Если хотим добавить remote, пишем 

$ git remote add remote_name remote_url

При клонировании автоматически конфигурируется origin

Флаги
- -f -- сразу сделать fetch
- rename old new -- переименовать remote
- rm == remove
- show -- информация о remote

In [ ]:
!git remote show origin

In [ ]:
!git remote add another_remote ../folder

In [ ]:
!git remote -v

In [ ]:
!git remote rm another_remote
!git remote -v

## git fetch

Загружает коммиты и ветки с указаных remotes

Если ничего не указать, то обновления текущей ветки загрузятся с origin

In [ ]:
os.chdir('../folder')
with open('file', 'a') as file:
    file.write('text\n')
!git add file
!git commit -m "added file"

In [ ]:
os.chdir('../folder_local')
with open('file', 'a') as file:
    file.write('text_local\n')
!git add file
!git commit -m "added file"

In [ ]:
!git fetch

In [ ]:
!git status

In [ ]:
!git branch

Что-то не видно remote веток. Чтобы их увидеть, добавим флаг -a == -all

In [ ]:
!git branch --all

In [ ]:
!git merge origin/HEAD

идем делаем merge...

- \$ vim file
- делаем merge
- \$ git add file
- \$ git commit -m "fixed merge issues"

In [ ]:
!git diff HEAD~1

Флаги:

- --all -- fetch all remotes
- --dry-run
- --prune -- удалить ветки, которых нет на remote

## git pull

Сокращение от git fetch + git merge

Чтобы вместо merge сделать rebase, нужно передать флаг --rebase

## git push

Добавляем локальные изменения в remote

$ git push remote branch

если вызвать git push, то git
<ol>
  <li> [нет remote] Посмотрит, на какой remote загружать, в переменнй branch.name.remote для текущей ветки. Если такой нет, то берется origin.</li>
  <li> [нет веток] Затем смотрит, что пушить, в переменной remote.name.push. Если ничего не нашел, то смотрит push.default. Если ее тоже нет, то считается, что push.default=simple, и git пытается запушить текущую ветку (tracking branch) в upstream branch, если их названия совпадают  </li>
</ol>

Флаги:
- --all -- запушить все ветки
- --prune -- удалить ветки из remote, которые не существуют более на локальной машине



На самом деле, команда чуть более хитрая. Что будет, если сделаем так:

$ git push origin :master

Правильно, удалиться ветка master из origin! Но лучше вызывать

$ git push origin --delete master

Вот синтаксис:

$ git push somewhere [+]\*src:dst

Причем src и dst могут быть пустыми.


- src -- имя ветки, которую хотим запушить (можно всякие master~n)
- :dst -- имя ветки, которую хотим обновить на remote (если не указана, то попробует обновить src на remote)
- \+ перед src -- если обновление dst не будет fast-forward, то добавление плюсика все равно заставит обновить dst (без плюсика гит ругнется) (ОСТОРОЖНО, тут можно накосячить. Почему?)

# Кастомизация git



Что делать, если хочется красивый вывод diff'a? Или если не хочется, чтобы файлы с логом загружались в репозиторий? Или хочется писать коммитты в любимом редакторе? Здесь речь пойдет о настройке окружения git и кастомизации его поведения.

## git config

Это представляет собой обычный конфигурационный файл, в котором записаны такие вещи, как ваше имя, почта, ваш любимый редактор и тп...

Существует три уровня иерархии:
- /etc/gitconfig -- конфиг для всех пользователей локальной машины (для windows этот файл лежит в C:\ProgramData\Git\config)
- ~/.gitconfig -- настройки для специфичного пользователя (для windows это C:\Users\$USER/.gitconfig)
- .git/config -- специфичные настройки для проекта

Переменные из нижнего уровня имеют высший приоритет перед таковыми из верхнего

Получим текущее состояние конфига:

In [ ]:
!git config --list

#### Чтобы посмотреть конфиг для системы, юзера, репозитория, можно воспользоваться флагами --system, --global, --file path (или --local, что стоит по дефолту) соответственно

In [ ]:
!git config --system --list

In [ ]:
!git config --global --list

In [ ]:
!git config --file ../.git/config --list

#### Что будет, если поставить одну и ту же переменную на разных уровнях:

In [ ]:
!git config --global user.nick itisgrisha
!git config --file ../.git/config user.nick newgrisha

In [ ]:
!git config --list | grep -i nick

Встречается два раза в списке. А git использует последнее значение переменной:

In [ ]:
!git config user.nick

Если хочется понять, откуда в переменной это значение, можно это спросить у git'a

In [ ]:
!git config --show-origin user.nick

### фишки конфига:

#### создание aliases

In [ ]:
!git config --replace-all alias.a "add --all"
!git config --replace-all alias.st "status -s"

In [ ]:
!git config --local --list

In [ ]:
!git st

#### commit.template -- указываем шаблон для сообщения коммита

$ git config --global commit.template path-to-template.txt


#### в зависимости от ОС могут быть ненужные файлы, специфичные для этой системы. Их можно исключить следующей переменной:

$ git config --global core.excludesfile path-to-gitignore.txt 

### в конфиге можно менять цвет вывода, указывать программку для просмотра diff-ов, и многое другое

### полезный файл с конфигом лежит [тут](https://gist.github.com/pksunkara/988716)

## gitattributes

Иногда некоторые настройки хочется применить только к специфичной папке проекта. Такие настройки называются аттрибутами. Также с ними можно указать, как мерджить определенные типы файлов, как смотреть diff для нетекстовых файлов, или что делать с разными символами переноса каретки

Есть две версии конфига:
- .gitattributes в рабочей директории -- если нужно шарить между проектами
- .git/info/attributes -- если не хотим шарить


На самом деле, почти все, что можно делать в аттрибутах, можно запихнуть в конфиг. Но лучше будет, если в конфиг писать общие настройки для пользователя\проекта, а в аттрибуты -- специфичные настройки (аттрибуты) для определенных файлов.

Указываем, что файл бинарный:

*.foo binary

Теперь git не будет пытаться считать дифф между этими файлами, или конвертировать переводы каретки

In [ ]:
# пишем в аттрибуты
with open('../.gitattributes', 'w') as file:
    file.write('*.foo binary\n')

In [ ]:
# добавляем бинарный файл
with open('../bar.foo', 'w') as file:
    file.write('foo\n')


!git add .
!git commit -m "added attributes and bar.foo"

In [ ]:
# модифицируем файл
with open('../bar.foo','a') as file:
    file.write('bar\n')

In [ ]:
!git diff

Скажем гиту, чтобы сам решал, что делать с crlf:

- \* text=auto

А виндовый файл пусть будет с crlf:

- *.some_windows_extension text eol=crlf

In [ ]:
with open('../.gitattributes', 'a') as file:
    file.write('* text=auto\n')

Допустим, что мы хотим, чтобы при коммите файл как-то обрабатывался. Пусть у нас есть программка tab2spaces, которая конвертирует табы в пробелы в текстовом файле. Тогда можно сделать такой фильтр:

>filter.t2s.clean tab2spaces


>filter.t2s.smudge cat

t2s -- название фильтра

Теперь скажем в аттрибутах, что хотим обрабатывать питоновские файлы перед коммитом:

>\*.py t2s

![](https://git-scm.com/book/en/v2/images/smudge.png)


### больше про аттрибуты [тут](https://git-scm.com/book/en/v2/Customizing-Git-Git-Attributes#filters_a)

## gitignore

Файл, в котором указано, какие файлы\папки нужно игнорировать (логи, мусор и тп).

Тут также есть иерархическая структура конфигов:

- переменная core.excludesFile
> паттерны, которые должны игнорироваться всегда (например, мусор от любимого редактора юзера)
- .git/info/exclude
> паттерны, специфичные для конкретного репозитория (например, на локальной машине)
- .gitignore (лежит в папке проекта вместе с .git)
> расшаренные паттерны, которые должны быть доступны всем, кто пользуется репозиторием (копируется с clone)
- Паттерны, переданные в команды, которые поддерживают игнорирование файлов


### Базовые правила написания фильтров:

- пустые строки и те, что начинаются с #, игнорируются (\#, чтобы включить хеш в паттерн)
- хвостовые пробелы игнорирутеся, если только перед ними не ставить "\"
- ! -- отрицание фильтра. Однако, нельзя включить файл, который лежит в исключенной директории. И вообще, git вообще не обращает внимания на файли в исключенных директориях, поэтому бессмысленно писать для них фильтры. Если очень хочется, то можно написать:

In [ ]:
#  !/foo
#  /foo/*
#  !/foo/bar

- foo/ -- директория foo (/bar/foo все еще включена!)
- \*\*/foo/ -- теперь foo/ игнорируется везде
- Если в паттерне нет слеша, то git будет игнорировать эти файлы везде (т е foo == \*\*/foo)
- foo/** -- все внутри foo
- a/\*\*/b -- любой уровень между a и b

### полезный [сборник](https://github.com/github/gitignore) фалов gitignore